# Jednostavni klasifikator slika

Ovaj bilježnik pokazuje kako klasificirati slike koristeći unaprijed treniranu neuronsku mrežu.

**Što ćete naučiti:**
- Kako učitati i koristiti unaprijed trenirani model
- Predobrada slika
- Predviđanje na slikama
- Razumijevanje rezultata pouzdanosti

**Primjena:** Identifikacija objekata na slikama (poput "mačka", "pas", "auto", itd.)

---


## Korak 1: Uvoz potrebnih biblioteka

Uvezimo alate koji su nam potrebni. Ne brinite ako još ne razumijete sve ovo!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Korak 2: Učitavanje unaprijed treniranog modela

Koristit ćemo **MobileNetV2**, neuronsku mrežu koja je već trenirana na milijunima slika.

Ovo se naziva **Transfer Learning** - korištenje modela koji je netko drugi trenirao!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Korak 3: Pomoćne funkcije

Napravimo funkcije za učitavanje i pripremu slika za naš model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Korak 4: Testiranje na uzorcima slika

Pokušajmo klasificirati neke slike s interneta!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klasificirajte svaku sliku


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Korak 5: Isprobajte vlastite slike!

Zamijenite URL ispod bilo kojim URL-om slike koju želite klasificirati.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Što se upravo dogodilo?

1. **Učitali smo unaprijed trenirani model** - MobileNetV2 je treniran na milijunima slika
2. **Predobradili smo slike** - Promijenili veličinu i formatirali ih za model
3. **Model je napravio predikcije** - Izračunao je vjerojatnosti za 1000 kategorija objekata
4. **Dekodirali smo rezultate** - Pretvorili brojeve u ljudima razumljive oznake

### Razumijevanje razina pouzdanosti

- **90-100%**: Vrlo pouzdano (gotovo sigurno točno)
- **70-90%**: Pouzdano (vjerojatno točno)
- **50-70%**: Djelomično pouzdano (možda točno)
- **Ispod 50%**: Nije jako pouzdano (neizvjesno)

### Zašto predikcije mogu biti netočne?

- **Neobičan kut ili osvjetljenje** - Model je treniran na tipičnim fotografijama
- **Više objekata** - Model očekuje jedan glavni objekt
- **Rijetki objekti** - Model poznaje samo 1000 kategorija
- **Slika niske kvalitete** - Zamućene ili pikselizirane slike su teže za analizu

---


## 🚀 Sljedeći koraci

1. **Isprobajte različite slike:**
   - Pronađite slike na [Unsplash](https://unsplash.com)
   - Desni klik → "Kopiraj adresu slike" za dobivanje URL-a

2. **Eksperimentirajte:**
   - Što se događa s apstraktnom umjetnošću?
   - Može li prepoznati objekte iz različitih kutova?
   - Kako se nosi s više objekata na slici?

3. **Saznajte više:**
   - Istražite [lekcije o računalnom vidu](../lessons/4-ComputerVision/README.md)
   - Naučite kako trenirati vlastiti klasifikator slika
   - Razumite kako funkcioniraju CNN-ovi (konvolucijske neuronske mreže)

---

## 🎉 Čestitamo!

Upravo ste izradili klasifikator slika koristeći najmoderniju neuronsku mrežu!

Ova ista tehnika pokreće:
- Google Photos (organiziranje vaših fotografija)
- Autonomna vozila (prepoznavanje objekata)
- Medicinsku dijagnostiku (analiza rendgenskih snimaka)
- Kontrolu kvalitete (otkrivanje nedostataka)

Nastavite istraživati i učiti! 🚀



---

**Izjava o odricanju odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za nesporazume ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
